# Demo to optimize kernel and stride in convolution operations

In [1]:
import sys
sys.path.append('..')
sys.path.append('../..')


import iterative_naive_nas

import utils.gen_dims_utils as gen_dim

from sklearn.model_selection import train_test_split

import tensorflow.compat.v1 as tf

import numpy as np

import custom_training

import GPyOpt

import utils.losses_utils as losses

import utils.data_utils as data_utils

import matplotlib.pyplot as plt

def get_models_and_shapes(eeg_file='../../optimized_nets/eeg/eeg_bold_shift_0.json', 
                          bold_file='../../optimized_nets/bold/bold_bold_shift_0.json',
                          decoder_file='../../optimized_nets/decoder/decoder_bold_shift_0.json'):
    json_file = open(eeg_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    eeg_network = tf.keras.models.model_from_json(loaded_model_json)
    
    json_file = open(bold_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    bold_network = tf.keras.models.model_from_json(loaded_model_json)
    
    json_file = open(decoder_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    decoder_network = tf.keras.models.model_from_json(loaded_model_json)
    
    return eeg_network, bold_network, decoder_network

/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/davidcalhas/anacond

Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/32/export/20130410320002_Segmentation_bin.vhdr...
Setting channel info structure...
Reading 0 ... 162022  =      0.000 ...   648.088 secs...
(30, 2607, 10)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/35/export/20130424350002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 197234  =      0.000 ...   788.936 secs...
(60, 2607, 10)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/36/export/20130425360002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 181949  =      0.000 ...   727.796 secs...
(90, 2607, 10)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/37/export/20130426370002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 195159  =      0.000 ...   780.636 secs...
(120, 2607, 10)
Extracting parameters from /home/davidcalhas

In [2]:
bold_shift=3
n_partitions=16
eeg_train, bold_train, eeg_val, bold_val = data_utils.load_data(list(range(10)), 
                                                                list(range(10, 12)), 
                                                                bold_shift=bold_shift, 
                                                                n_partitions=n_partitions, 
                                                                roi=1, roi_ica_components=20)
n_voxels = bold_train.shape[1]

print("Finished Loading Data")

X_train_eeg, X_train_bold, tr_y = data_utils.create_eeg_bold_pairs(eeg_train, bold_train)
X_val_eeg, X_val_bold, tv_y = data_utils.create_eeg_bold_pairs(eeg_val, bold_val)


X_train_eeg = X_train_eeg.astype(np.float32)
X_train_bold = X_train_bold.astype(np.float32)
X_val_eeg = X_val_eeg.astype(np.float32)
X_val_bold = X_val_bold.astype(np.float32)


tr_y = np.array(tr_y, dtype=np.float32)
tv_y = np.array(tv_y, dtype=np.float32)

eeg_train = eeg_train.astype('float32')
bold_train = bold_train.astype('float32')
eeg_val = eeg_val.astype('float32')
bold_val = bold_val.astype('float32')

print("Pairs Created")

Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/32/export/20130410320002_Segmentation_bin.vhdr...
Setting channel info structure...
Reading 0 ... 162022  =      0.000 ...   648.088 secs...
(16, 2607, 19)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/35/export/20130424350002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 197234  =      0.000 ...   788.936 secs...
(32, 2607, 19)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/36/export/20130425360002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 181949  =      0.000 ...   727.796 secs...
(48, 2607, 19)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/37/export/20130426370002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 195159  =      0.000 ...   780.636 secs...
(64, 2607, 19)
Extracting parameters from /home/davidcalhas/

In [3]:
eeg_network, bold_network, decoder_network = get_models_and_shapes()

eeg_input_shape = (64, 5, 19, 1)
bold_input_shape=(2570, 19, 1)

multi_modal_model = custom_training.multi_modal_network(eeg_input_shape, bold_input_shape, eeg_network, bold_network, dcca=False)

### Optimize the kernel and stride of the current shapes

In [4]:
def batch_to_dim(shape):
    return shape[1:]


hyperparameters = []

def add_kernel_stride_hyperparameters(network, net_name):
    
    hyperparameters = []
    
    previous_layer = -1
    
    for layer in range(len(network.layers)):
        print(network.layers[layer].name)
        if('transpose' in network.layers[layer].name):
            if(previous_layer < 0):
                eeg_input_shape = (64, 5, 19, 1)
            else:
                eeg_input_shape = batch_to_dim(network.layers[previous_layer].output_shape)
            print(layer, eeg_input_shape)
            for dim in range(len(eeg_input_shape)):
                print(tuple(gen_dim.get_possible_kernel_size_deconv(eeg_input_shape[dim], batch_to_dim(network.layers[layer].output_shape)[dim])))
                hyperparameters += [{'name': net_name + '_' + str(layer) + '_' + str(dim), 'type': 'discrete',
                                    'domain': tuple(gen_dim.get_possible_kernel_size_deconv(eeg_input_shape[dim], batch_to_dim(network.layers[layer].output_shape)[dim]))}]

            previous_layer = layer
        
        elif('transpose' not in network.layers[layer].name and 'reshape' not in network.layers[layer].name):
            if(previous_layer < 0):
                eeg_input_shape = (2570, 19, 1)
            else:
                eeg_input_shape = batch_to_dim(network.layers[previous_layer].output_shape)
            print(layer, eeg_input_shape)
            for dim in range(len(eeg_input_shape)):
                print(tuple(gen_dim.get_possible_kernel_size_deconv(eeg_input_shape[dim], batch_to_dim(network.layers[layer].output_shape)[dim])))
                hyperparameters += [{'name': net_name + '_' + str(layer) + '_' + str(dim), 'type': 'discrete',
                                    'domain': tuple(gen_dim.get_possible_kernel_size_deconv(eeg_input_shape[dim], batch_to_dim(network.layers[layer].output_shape)[dim]))}]

            previous_layer = layer
            
    return hyperparameters
            
hyperparameters += [add_kernel_stride_hyperparameters(eeg_network, 'eeg_net')]
hyperparameters += [add_kernel_stride_hyperparameters(bold_network, 'bold_net')]

conv3d_transpose_105
0 (64, 5, 19, 1)
((451, 1), (10, 8), (73, 7), (136, 6), (199, 5), (262, 4), (325, 3), (388, 2))
((1, 1),)
((1, 1),)
((1, 1),)
reshape_105
conv2d_105
0 (2570, 19, 1)


AttributeError: The layer "conv2d_105" has multiple inbound nodes, with different output shapes. Hence the notion of "output shape" is ill-defined for the layer. Use `get_output_shape_at(node_index)` instead.

In [ ]:
hyperparameters